# User filtering
특정 직업과 관련된 해시태그 검색 후, 해당 직업이 아닌 유저를 필터링하는 과정이 필요하다.

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [361]:
# 예제 데이터
df0 = [{"userid": "111111", "username": "yoouza", "tags":["중경", "경찰", "좋반", "팔로우"] , "freq" : {"중경": 13, "경찰": 10, "좋반": 12, "팔로우": 8}},
{"userid": "222222", "username": "orange", "tags":["중경", "경찰", "좋반", "경찰대학"], "freq" : {"중경": 8, "경찰": 9, "좋반": 10, "경찰대학": 12}}]
df0 = pd.DataFrame(df0)[['userid', 'username', 'tags', 'freq']]
df1 = [{"userid": "33333", "username": "enhahappy", "tags":["교사", "교생실습", "담임쌤", "좋반"] , "freq" : {"교사": 12, "교생실습": 10, "담임쌤": 6, "좋반": 5}},
{"userid": "44444", "username": "blueganet", "tags":["교생실습", "팔로우", "학생"], "freq" : {"교생실습": 12, "팔로우": 7, "학생": 6}}]
df1 = pd.DataFrame(df1)[['userid', 'username', 'tags', 'freq']]
df2 = [{"userid": "5555", "username": "kbs", "tags":["의사", "병원", "진료중", "좋반"] , "freq" : {"의사": 12, "병원": 10, "진료중": 6, "좋반": 5}},
{"userid": "6666", "username": "hotte", "tags":["의사", "간호사", "팔로우해용"], "freq" : {"의사": 12, "간호사": 7, "팔로우해용": 6}}]
df2 = pd.DataFrame(df2)[['userid', 'username', 'tags', 'freq']]

In [362]:
print('제거 전: 경찰, 교사, 의사')
display(df0)
display(df1)
display(df2)

제거 전: 경찰, 교사, 의사


,userid,username,tags,freq
0,111111,yoouza,"[중경, 경찰, 좋반, 팔로우]","{'중경': 13, '경찰': 10, '좋반': 12, '팔로우': 8}"
1,222222,orange,"[중경, 경찰, 좋반, 경찰대학]","{'중경': 8, '경찰': 9, '좋반': 10, '경찰대학': 12}"


,userid,username,tags,freq
0,33333,enhahappy,"[교사, 교생실습, 담임쌤, 좋반]","{'교사': 12, '교생실습': 10, '담임쌤': 6, '좋반': 5}"
1,44444,blueganet,"[교생실습, 팔로우, 학생]","{'교생실습': 12, '팔로우': 7, '학생': 6}"


,userid,username,tags,freq
0,5555,kbs,"[의사, 병원, 진료중, 좋반]","{'의사': 12, '병원': 10, '진료중': 6, '좋반': 5}"
1,6666,hotte,"[의사, 간호사, 팔로우해용]","{'의사': 12, '간호사': 7, '팔로우해용': 6}"


### 검색 해시태그 제거

In [363]:
# 각 직업별 검색 해시태그
hashtags = [["경찰", "중경"], ["교사"], ["의사"]]

def del_tags(tags, hashtag):
    lst = []
    for tag in tags:
        if tag not in hashtag:
            lst.append(tag)
    return lst

def del_freq(freq, hashtag):
    dic = {}
    for (w, i) in freq.items():
        if w not in hashtag:
            dic[w] = i
    return dic

def del_hashtag(num_df, hashtag):
    df = globals()['df{}'.format(num_df)]
    tags = df.tags.apply(lambda x: del_tags(x, hashtag))
    freq = df.freq.apply(lambda x: del_freq(x, hashtag))
    df['tags'] = tags
    df['freq'] = freq
    return df

for i in range(N_category):
    globals()['df{}'.format(i)] = del_hashtag(i, hashtags[i])

In [364]:
print('제거 후: 경찰, 교사, 의사')
display(df0)
display(df1)
display(df2)

제거 후: 경찰, 교사, 의사


,userid,username,tags,freq
0,111111,yoouza,"[좋반, 팔로우]","{'좋반': 12, '팔로우': 8}"
1,222222,orange,"[좋반, 경찰대학]","{'좋반': 10, '경찰대학': 12}"


,userid,username,tags,freq
0,33333,enhahappy,"[교생실습, 담임쌤, 좋반]","{'교생실습': 10, '담임쌤': 6, '좋반': 5}"
1,44444,blueganet,"[교생실습, 팔로우, 학생]","{'교생실습': 12, '팔로우': 7, '학생': 6}"


,userid,username,tags,freq
0,5555,kbs,"[병원, 진료중, 좋반]","{'병원': 10, '진료중': 6, '좋반': 5}"
1,6666,hotte,"[간호사, 팔로우해용]","{'간호사': 7, '팔로우해용': 6}"


### 전체 빈도수
df뒤의 숫자와 1:1 매칭


In [384]:
N_category = 3 # 직업 카테고리 수

for i in range(N_category):
    globals()['freq_whole_{}'.format(i)] = Counter()
    for f in globals()['df{}'.format(i)].freq:
        globals()['freq_whole_{}'.format(i)] += Counter(f)

In [366]:
freq_whole_0, freq_whole_1, freq_whole_2

(Counter({'좋반': 22, '팔로우': 8, '경찰대학': 12}),
 Counter({'교생실습': 22, '담임쌤': 6, '좋반': 5, '팔로우': 7, '학생': 6}),
 Counter({'병원': 10, '진료중': 6, '좋반': 5, '간호사': 7, '팔로우해용': 6}))

### 새로운 voca dict 생성
단어의 등장 문서수가 직업군의 절반을 넘을 경우, 해당 단어를 제거한 새로운 딕셔너리 생성(각 직업군에 대표성 있다고 판단)

In [367]:
for i in range(N_category):
    globals()['voca{}'.format(i)] = {}
    globals()['freq_other{}'.format(i)] = {}

for num in range(N_category):
    for (w, n) in globals()['freq_whole_{}'.format(num)].items():
        w_freq = 1
        for i in range(N_category):
            if i != num:
                if w in globals()['freq_whole_{}'.format(i)]:
                    w_freq += 1
        globals()['freq_other{}'.format(num)][w] = w_freq
        if w_freq <= N_category//2:
            globals()['voca{}'.format(num)][w] = n

In [368]:
# 각 직업의 대표성을 띠는 단어들
voca0, voca1, voca2

({'경찰대학': 12},
 {'교생실습': 22, '담임쌤': 6, '학생': 6},
 {'병원': 10, '진료중': 6, '간호사': 7, '팔로우해용': 6})

In [369]:
# 등장한 문서 수
freq_other0, freq_other1, freq_other2

({'좋반': 3, '팔로우': 2, '경찰대학': 1},
 {'교생실습': 1, '담임쌤': 1, '좋반': 3, '팔로우': 2, '학생': 1},
 {'병원': 1, '진료중': 1, '좋반': 3, '간호사': 1, '팔로우해용': 1})

In [370]:
N = 2 # 상위 몇 개를 뽑아낼지 지정
for i in range(N_category):
    globals()['voca{}_top'.format(i)] = dict(Counter(globals()['voca{}'.format(i)]).most_common(N))

In [371]:
# 상위 N개의 voca dict
voca0_top, voca1_top, voca2_top

({'경찰대학': 12}, {'교생실습': 22, '담임쌤': 6}, {'병원': 10, '간호사': 7})

### negative dict 생성
타 직업군의 voca를 다 더한다.

In [372]:
for i in range(N_category):
    globals()['neg{}'.format(i)] = Counter()

for i in range(N_category):
    for j in range(N_category):
        if j != i:
            globals()['neg{}'.format(i)] += Counter(globals()['voca{}_top'.format(j)])

In [373]:
# 타 직업군에서의 단어 목록
neg0, neg1, neg2

(Counter({'교생실습': 22, '담임쌤': 6, '병원': 10, '간호사': 7}),
 Counter({'경찰대학': 12, '병원': 10, '간호사': 7}),
 Counter({'경찰대학': 12, '교생실습': 22, '담임쌤': 6}))

## neg voca에 많이 등장할수록 중요도를 낮추는 방법 고찰
- 목표: 다른 직업군을 대표하는 단어가 등장할수록 0에 가깝고 그 외의 경우에는 1에 가깝게
1. log값 취하고 1더한 후 역수
2. 상-하 뒤집은 후 x축으로 +N_sigmoid만큼 평행이동한 sigmoid 적용(0~1사이 값, 변곡점은 N_sigmoid에 위치)


### 1. negative voca에 log값 취하기

In [374]:
for i in range(N_category):
    globals()['neg{}_log'.format(i)] = {}

for i in range(N_category):
    for (w, n) in globals()['neg{}'.format(i)].items():
        globals()['neg{}_log'.format(i)][w] = np.log(n)

In [375]:
neg0_log, neg1_log, neg2_log

({'교생실습': 3.091042453358316,
  '담임쌤': 1.791759469228055,
  '병원': 2.302585092994046,
  '간호사': 1.9459101490553132},
 {'경찰대학': 2.4849066497880004,
  '병원': 2.302585092994046,
  '간호사': 1.9459101490553132},
 {'경찰대학': 2.4849066497880004,
  '교생실습': 3.091042453358316,
  '담임쌤': 1.791759469228055})

### 2. negative voca에 sigmoid 적용하기
- N_sigmoid는 neg 단어의 빈도수 보고 조정해야 함
- 상위 N개의 평균 값 정도가 적절할듯

In [376]:
def mysigmoid(x, N_sigmoid):
    return 1 / (1 + np.exp(x - N_sigmoid))

for i in range(N_category):
    globals()['neg{}_sig'.format(i)] = {}

for i in range(N_category):
    for (w, n) in globals()['neg{}'.format(i)].items():
        globals()['neg{}_sig'.format(i)][w] = mysigmoid(n, N_sigmoid = 20)

In [377]:
neg0_sig, neg1_sig, neg2_sig

({'교생실습': 0.11920292202211755,
  '담임쌤': 0.9999991684719722,
  '병원': 0.9999546021312976,
  '간호사': 0.999997739675702},
 {'경찰대학': 0.9996646498695336,
  '병원': 0.9999546021312976,
  '간호사': 0.999997739675702},
 {'경찰대학': 0.9996646498695336,
  '교생실습': 0.11920292202211755,
  '담임쌤': 0.9999991684719722})

# 최종 scoring
- 주부, 백수는 계산할 필요 없음
- negative 변수를 곱한 tf-idf 중요도 사용
- tf-idf에 곱해줄 보정 변수 neg 계산법
1. log: 1/(1+neg_log)
2. sigmoid: neg_sig 그대로 사용

In [378]:
for i in range(N_category):
    globals()['df{}'.format(i)]['score_log'] = 0
    globals()['df{}'.format(i)]['score_sigmoid'] = 0

In [379]:
def scoring(freq_row, num_df, method):
    score = 0
    for (w, n) in freq_row.items():
        tf = n
        idf = N_category / (1 + globals()['freq_other{}'.format(num_df)][w])
        try:
            if method == 'log':
                neg = 1 / (1 + globals()['neg{}_log'.format(num_df)][w])
            elif method == 'sigmoid':
                neg = globals()['neg{}_sig'.format(num_df)][w]
        except:
            neg = 1
        score += tf * idf * neg
    return score

In [380]:
# 실제로 돌릴때는 N_category-2 (주부, 백수)

for i in range(N_category):
    globals()['df{}'.format(i)]['score_log'] = globals()['df{}'.format(i)]['freq'].apply(lambda x: scoring(x, i, method = 'log'))
    globals()['df{}'.format(i)]['score_sigmoid'] = globals()['df{}'.format(i)]['freq'].apply(lambda x: scoring(x, i, method = 'sigmoid'))

In [382]:
print('예제 데이터는 score 비교 불가능..')
display(df0)
display(df1)
display(df2)

예제 데이터는 score 비교 불가능..


,userid,username,tags,freq,score_log,score_sigmoid
0,111111,yoouza,"[좋반, 팔로우]","{'좋반': 12, '팔로우': 8}",17.0,17.0
1,222222,orange,"[좋반, 경찰대학]","{'좋반': 10, '경찰대학': 12}",25.5,25.5


,userid,username,tags,freq,score_log,score_sigmoid
0,33333,enhahappy,"[교생실습, 담임쌤, 좋반]","{'교생실습': 10, '담임쌤': 6, '좋반': 5}",27.75,27.75
1,44444,blueganet,"[교생실습, 팔로우, 학생]","{'교생실습': 12, '팔로우': 7, '학생': 6}",34.00,34.00


,userid,username,tags,freq,score_log,score_sigmoid
0,5555,kbs,"[병원, 진료중, 좋반]","{'병원': 10, '진료중': 6, '좋반': 5}",27.75,27.75
1,6666,hotte,"[간호사, 팔로우해용]","{'간호사': 7, '팔로우해용': 6}",19.50,19.50
